In [1]:
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import os
import numpy as np
import uuid
import random

In [57]:
def add_background_qr(qr_tag):
    tag_back = np.zeros((128, 128, 4))
    mask = np.zeros((128, 128))
    x_start = random.randint(50,60)
    y_start = random.randint(50,60)
    tag_back[x_start: x_start+10, y_start: y_start+10, :] = qr_tag
    mask[x_start: x_start+10, y_start: y_start+10] = np.full((10, 10), 255)
    tag_back = tag_back.astype(np.uint8)
    mask = mask.astype(np.uint8)
    return (Image.fromarray(tag_back), Image.fromarray(mask, mode='L'))

In [58]:
# mask = np.zeros((128, 128))
# mask[0: 10, 0: 10] = np.full((10, 10), 255)
# mask = mask.astype(np.uint8)
# img = Image.fromarray(mask, mode='L')
# plt.imshow(img, cmap='gray')

In [63]:
qr_tags_path = '/Users/chamathabeysinghe/Projects/monash/qr-elimination/data/qr_embeded_dataset/qr_tags'
qr_tags = [Image.open(path) for path in [os.path.join(qr_tags_path, x)  for x in os.listdir(qr_tags_path) if not x.startswith('.')]]
for qr_tag in qr_tags:
    qr_tag.thumbnail((10, 10), Image.ANTIALIAS)
# qr_tags = [add_background_qr(qr_tag) for qr_tag in qr_tags]

In [64]:
crop_imgs_path = '/Users/chamathabeysinghe/Projects/monash/qr-elimination/data/qr_embeded_dataset/ant_cropped_images'
img_paths = [os.path.join(crop_imgs_path, x) for x in os.listdir(crop_imgs_path) if not x.startswith('.')]
dataset_X = '/Users/chamathabeysinghe/Projects/monash/qr-elimination/data/qr_embeded_dataset/dataset/X'
dataset_masks = '/Users/chamathabeysinghe/Projects/monash/qr-elimination/data/qr_embeded_dataset/dataset/masks'
dataset_Y = '/Users/chamathabeysinghe/Projects/monash/qr-elimination/data/qr_embeded_dataset/dataset/Y'
os.makedirs(dataset_X, exist_ok=True)
os.makedirs(dataset_Y, exist_ok=True)
os.makedirs(dataset_masks, exist_ok=True)

In [65]:
for p in img_paths:
    img = Image.open(p)
    img = img.convert("RGBA") 
    if (img.size != (128, 128)):
        continue
    for qr_tag in qr_tags:
        filename = str(uuid.uuid4())+'.png'
        qr_image, mask = add_background_qr(qr_tag)
        embedded_image = Image.alpha_composite(img, qr_image)
        embedded_image.save(os.path.join(dataset_X, filename))
        img.save(os.path.join(dataset_Y, filename))
        mask.save(os.path.join(dataset_masks, filename))
    